In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY =os.environ.get("GOOGLE_API_KEY")
GROQ_API_KEY =os.environ.get("GROQ_API_KEY")
TAVILIY_API_KEY=os.environ.get("TAVILY_API_KEY")

In [4]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="groq:openai/gpt-oss-120b")
model = init_chat_model(model="google_genai:gemini-2.5-flash-lite")

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [4]:
from pprint import pprint
pprint(tools)

[StructuredTool(name='search-flight', description='\n# Search for a flight\n\n## Description\n\nUses the Kiwi API to search for available flights between two locations on a specific date.\n\n## How it works\n\nThe tool will:\n1. Search for matching locations to resolve airport codes\n2. Find available flights for the specified route and date range\n\n## Method\n\nCall this tool whenever a user wants to search for flights, regardless of whether they provided exact airport codes or just city names.\n\nYou should display the returned results in a markdown table format: Group the results by price (those who are the cheapest), duration (those who are the shortest, i.e. have the smallest \'totalDurationInSeconds\') and the rest (those that could still be interesting).\n\nAlways display for each flight in order:\n  - In the 1st column: The departure and arrival airports, including layovers (e.g. "Paris CDG → Barcelona BCN → Lisbon LIS")\n  - In the 2nd column: The departure and arrival dates 

In [3]:
from typing import Dict, Any
from tavily import TavilyClient
from langchain.tools import tool

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [6]:
from langchain.agents import AgentState

class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    #genre: str

In [7]:
from langchain.agents import create_agent

# Travel agent
travel_agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="""
    You are a travel agent. Search for flights to the desired destination wedding location.
    You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options.
    """
)

In [8]:
# Venue agent
venue_agent = create_agent(
    model=llm,
    tools=[web_search],
    system_prompt="""
    You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options.
    """
)

In [9]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command

@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travel agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    response = await travel_agent.ainvoke({"messages": [HumanMessage(content=f"Find flights from {origin} to {destination}")]})
    return response['messages'][-1].content

@tool
def search_venues(runtime: ToolRuntime) -> str:
    """Venue agent chooses the best venue for the given location and capacity."""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

# @tool
# def suggest_playlist(runtime: ToolRuntime) -> str:
#     """Playlist agent curates the perfect playlist for the given genre."""
#     genre = runtime.state["genre"]
#     query = f"Find {genre} tracks for wedding playlist"
#     response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
#     return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, runtime: ToolRuntime) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count"""
    return Command(update={
        "origin": origin, 
        "destination": destination, 
        "guest_count": guest_count, 
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )

In [10]:
from langchain.agents import create_agent

coordinator = create_agent(
    model=llm,
    tools=[search_flights, search_venues, update_state],
    state_schema=WeddingState,
    system_prompt="""
    You are a wedding coordinator. Delegate tasks to your specialists for flights, and venues.
    First find all the information you need to update the state. Once that is done you can delegate the tasks.
    Once you have received their answers, coordinate the perfect wedding for me.
    """
)

In [ ]:
from langchain.messages import HumanMessage

response = await coordinator.ainvoke(
    {
        "messages": [HumanMessage(content="I'm from Pakistan and I'd like a wedding in Karachi for 100 guests")],
    })

In [14]:
print(response['messages'][-1].content)

Great! I’ve recorded the basics:

- **Origin:** Pakistan  
- **Destination:** Karachi  
- **Guest count:** 100  

To move forward and make sure everything is perfectly tailored, could you let me know a few more details?

1. **Travel**  
   - Are any of your guests traveling from outside Karachi (or outside Pakistan)? If so, please list the cities/countries they’ll be coming from and any preferred travel dates.  
   - Do you have a preferred airline class (economy, business, etc.) or any special travel requirements?

2. **Venue**  
   - What type of venue are you envisioning (e.g., hotel ballroom, garden, beachside, historic mansion, etc.)?  
   - Do you have a budget range per guest or overall for the venue?  
   - Any specific amenities you’d like (e.g., on‑site catering, parking, accommodation for guests, indoor/outdoor spaces)?

3. **Date & Timing**  
   - Do you have a wedding date (or month) in mind?  
   - Will the event be a single‑day ceremony/reception or a multi‑day celebrati

In [ ]:
print(response["messages"][-1].content)

Great! I’ve recorded the basics:

- **Origin:** London  
- **Destination:** Paris  
- **Guest count:** 100  

To move forward, I’ll need a bit more detail:

1. **Preferred wedding date (or at least month)** – this will help me find the best flight options and venue availability.  
2. **Any specific flight preferences** (e.g., direct flights only, preferred airlines, budget range).  
3. **Venue style or requirements** (e.g., indoor vs. outdoor, historic building, garden, ballroom, accessibility needs).  

Once I have those details, I’ll ask my flight and venue specialists to start searching and then bring everything together for you. 🌟
